# ポートフォリオ最適化

このチュートリアルではポートフォリオ最適化を扱います。ポートフォリオとは株式や債券などの金融商品の組み合わせのことです。このような金融商品は銀行預金とは異なり、将来の受取金額が確定していないリスク資産であり、収益率も不確定です。このようなリスク資産への投資においては、適切なポートフォリオを組み、互いに大きな相関のない複数資産へ分散投資することが推奨されます。こうすることで、保有するある資産の価値が大きく下がった場合においても、その資産の値動きと大きな相関を持たない別の資産によってカバーできる可能性があります。

一般的に、分散投資におけるリスクとリターンにはトレードオフの関係があります。収益率のバラつきや損失を被るリスクを抑え、リターンを最大化するには、このトレードオフのバランスを適切に反映したポートフォリオを組む必要があります。

ここでは、Fixstars Amplify を用いて、ポートフォリオの最適化を行います。最適化にはヒストリカルデータ方式に基づく推計値を用います。ヒストリカルデータ方式とは、過去の値動きデータに基づいて資産の期待収益率やリスクを求める推計方法です。本サンプルプログラムでは、

- [ヒストリカルデータの取得](#1)
- [最適ポートフォリオ求解のための定式化及びソルバの実装](#2)
- [最適化の実行と評価](#3)
- [運用シミュレーション](#4)

と、順を追って解説します。

※本サンプルプログラムは、Fixstars Amplify を利用した最適化アプリケーションのデモンストレーションを目的としています。本サンプルプログラムで提供される情報を基に、実際の運用を行う際には、ユーザーご自身の責任において実施してください。


<a id="1"></a>

## ヒストリカルデータの取得

まず、ヒストリカルデータ取得・利用のためのクラス `HistoricalDataLoader` を実装します。`HistoricalDataLoader` は、あらかじめ与えられた運用銘柄や運用日、運用期間、サンプリング期間などといった情報をもとに、保存データから必要な期間の情報を取得します。保存データに含まれない期間を考慮する必要がある場合、新規データ取得には、`pandas_datareader` を用い、[Stooq](https://stooq.com/) のデータベースから対象となる銘柄のヒストリカルデータを取得します。


In [ ]:
import datetime
import pathlib
import pandas as pd
from pandas_datareader import data

ONE_DAY = datetime.timedelta(days=1)


class HistoricalDataLoader:

    def __init__(
        self,
        tickers: list[str],  # 考慮する銘柄のティッカーのリスト
        operation_start_date: datetime.date,  # 運用開始日
        cache_dir: pathlib.Path,  # 保存データの保存先ディレクトリパス
        num_days_sampling=10,  # デフォルトでは、ヒストリカルデータ 10 営業日分
        num_days_operation=5,  # デフォルトでは、運用期間は 5 営業日
        use_cache: bool = True,  # 全てのデータをダウンロードするには、False（株式分割の場合、Stooq では、過去に渡って株価が調整されるため）
        stdout: bool = False,  #  標準出力する場合は True
    ):
        self._tickers = tickers
        self._num_days_sampling = num_days_sampling
        self._num_days_operation = num_days_operation
        self._operation_start_date = operation_start_date
        self._cache_dir = cache_dir
        self._stdout = stdout
        self._history = self._construct_history(use_cache)
        self._sample_end_date: None | datetime.date = None
        self._operation_end_date: None | datetime.date = None

    def _construct_history(self, use_cache: bool) -> pd.DataFrame:
        """全銘柄のヒストリカルデータを構築する関数"""
        history = pd.DataFrame()
        for ticker in self._tickers:
            individual_history = self._fetch_save_individual_history(ticker, use_cache)
            history = pd.concat([history, individual_history], axis=1).reindex(
                index=individual_history.index
            )
        self._print_deletion_periods(history)
        # 全銘柄が取引されていない日（営業日以外）のデータを削除して返却
        return history.dropna(how="any")

    def _print_deletion_periods(self, history: pd.DataFrame) -> None:
        """全銘柄が取引されていない日（営業日以外）をプリントする関数"""
        mask = history.isnull().any(axis=1)
        i = 1
        while i < len(mask) - 1:
            if mask.iloc[i] and not mask.iloc[i - 1]:
                del_start_date = mask.index[i].to_pydatetime().date()  # type: ignore
                del_end_date = ""
                for j in range(i, len(mask) - 1):
                    if not mask.iloc[j + 1]:
                        del_end_date = mask.index[j].to_pydatetime().date()  # type: ignore
                        i = j
                        break
                if self._stdout:
                    print(f"deletion {del_start_date}-{del_end_date}, nan is found.")
            i += 1

    def _calc_sample_end_date(self) -> datetime.date:
        """ヒストリカルデータのサンプリングの終了日を計算"""
        check_date = self._operation_start_date
        # サンプリング終了日は、運用開始日より前の運用開始日に最も近い営業日。
        # 運用開始日の前日から過去へ 14 日遡れば、営業日が少なくとも1つ見つかるはず（例：旧正月前後の中国株）
        for _ in range(14):
            check_date -= ONE_DAY
            key = check_date.strftime("%Y-%m-%d")
            if key in self._history.index:
                return check_date
        raise RuntimeError(f"could not find a valid end date. {check_date=}")

    def _calc_operation_end_date(self) -> datetime.date:
        """ヒストリカルデータ内に運用期間を含むデータがある場合、運用期間終了日を計算"""
        hist = self.extract_operation_period()
        if len(hist) == 0:
            raise RuntimeError("historical data do not include operation period.")
        return hist.index[0].to_pydatetime().date()  # type: ignore

    def _exist_operation_start_date(self) -> bool:
        """ヒストリカルデータ内に運用開始日のデータが含まれるか否か"""
        return self._operation_start_date.strftime("%Y-%m-%d") in self._history.index

    def exist_operation_duration(self) -> bool:
        """ヒストリカルデータ内に運用期間全てのデータが含まれるか否か"""
        if not self._exist_operation_start_date():
            return False
        date_count = 0
        date = self._operation_start_date
        while date_count < self._num_days_operation:
            if date.strftime("%Y-%m-%d") in self._history.index:
                date_count += 1
            date += ONE_DAY
            if date > self._history.index[0].to_pydatetime().date():  # type: ignore
                return False
        return True

    @property
    def num_days_sampling(self) -> int:
        """サンプリング期間（日数）を返す"""
        return self._num_days_sampling

    @property
    def num_days_operation(self) -> int:
        """運用期間（日数）を返す"""
        return self._num_days_operation

    @property
    def tickers(self) -> list[str]:
        """考慮する銘柄のティッカーのリストを返す"""
        return self._tickers

    @property
    def history(self) -> pd.DataFrame:
        """全期間のヒストリカルデータを返す"""
        return self._history

    @property
    def operation_end_date(self) -> datetime.date:
        """ヒストリカルデータ内に運用期間を含むデータがある場合、運用期間終了日を返却"""
        if self._operation_end_date is None:
            self._operation_end_date = self._calc_operation_end_date()
        return self._operation_end_date

    @property
    def sample_end_date(self) -> datetime.date:
        """サンプリング終了日を返却"""
        if self._sample_end_date is None:
            self._sample_end_date = self._calc_sample_end_date()
        return self._sample_end_date

    def _load_cache(self, filepath: pathlib.Path, num_rows: int | None = None):
        """保存済みのヒストリカルデータをロード"""
        hist = pd.read_csv(filepath, index_col=0, nrows=num_rows)
        hist.index = pd.to_datetime(hist.index)  # type: ignore
        if self._stdout:
            print(f"Cache: {filepath.name} from {hist.index[-1]} to {hist.index[0]}")  # type: ignore
        return hist

    def _download_history(self, ticker: str, start_date: datetime.date | None = None):
        """Stooq から start_date 以降のヒストリカルデータをダウンロード"""
        if self._stdout:
            print(f"Download: {ticker}, from {start_date}")
        hist = data.DataReader(ticker, "stooq", start_date)
        if len(hist) == 0:
            return pd.DataFrame()
        else:
            return hist.drop(["Open", "High", "Low", "Volume"], axis=1)

    def _save_cache(self, df: pd.DataFrame, filepath: pathlib.Path):
        """ヒストリカルデータを保存"""
        df.index.name = None
        df.to_csv(filepath)

    def _fetch_save_individual_history(
        self, ticker: str, use_cache: bool = True
    ) -> pd.DataFrame:
        """個別銘柄のヒストリカルデータを取得・保存"""
        filepath = self._cache_dir.joinpath(ticker + ".csv")
        start_date: datetime.date | None = None
        hist = pd.DataFrame()
        if filepath.exists() and use_cache:
            hist = self._load_cache(filepath)
            # ダウンロード必要な期間の開始日
            start_date = hist.index[0].date() + ONE_DAY  # type: ignore
        # +2 は週末を考慮するためのバッファー
        delta_days = datetime.timedelta(days=self._num_days_operation + 2)
        if start_date is None or start_date <= self._operation_start_date + delta_days:
            hist = pd.concat([self._download_history(ticker, start_date), hist])
            self._save_cache(hist, filepath)
        return hist.rename(columns={"Close": ticker})  # 終値のみを考慮

    def extract_sampling_period(self) -> pd.DataFrame:
        """サンプリング期間のヒストリカルデータを返却"""
        i_end = self._history.index.get_loc(self.sample_end_date.strftime("%Y-%m-%d"))
        ret = self._history.iloc[i_end : i_end + self._num_days_sampling]  # type: ignore
        return ret

    def extract_operation_period(self) -> pd.DataFrame:
        """（ヒストリカルデータに含まれる場合）運用期間のヒストリカルデータを返却"""
        key = self._operation_start_date.strftime("%Y-%m-%d")
        i_start = self._history.index.get_loc(key) if key in self._history.index else 0
        if not self.exist_operation_duration():
            print("The historical data do not include the operation period.")
            return pd.DataFrame()
        ret = self._history.iloc[i_start - self._num_days_operation + 1 : i_start + 1]  # type: ignore
        return ret

<a id="2"></a>

## ポートフォリオ最適化の定式化及び実装

それではヒストリカルデータ方式による推計値を使い、最適なポートフォリオを実現するための定式化を行います。

まず、$n$ 個の金融商品を対象とし、$d$ 日間の運用を目的として、ポートフォリオを構成するとします。投資合計金額に対して金融商品 $i$ への投資比率を $w_i$ とすると、$w_i$ が満たすべき制約条件は、

$$
\sum_{i=1}^n w_i = 1
$$

となります。

各銘柄 $i$ の終値を $p_i$ としたとき、金融商品 $i$ の収益率 $r_i$ は、運用開始時の価額 $p_{i,s}$ と運用終了時の価額 $p_{i,e}$ を用いて、

$$
r_i = (p_{i,e} - p_{i,s}) / p_{i,s}
$$

と定義されます。また、このとき、ポートフォリオ全体の収益率 $r_p$ は、

$$
r_p = \sum_{i=1}^{n} r_i w_i
$$

となります。一方、分散投資におけるリスクとは、通常、ポートフォリオ全体収益率の分散 $\sigma^2_p$ や標準偏差 $\sigma_p$であり、これらは次のように記述することができます。

$$
\sigma_p^2 = \sum_{i=1}^n \sum_{j=1}^n w_i w_j \sigma_{i,j}.
$$

ここで、$\sigma_{i,j}$ は、金融商品 $i$ と $j$ の収益率の共分散です。一般的に、収益率 $r_p$ が大きく、リスク $\sigma_p^2$ が小さければ良いポートフォリオとされます。このことから、以下の平均・分散モデル $f(w_i)$ を目的関数とし、これを最小化するように最適化を行います。

$$
f(w_i) = - r_p + \frac{\gamma}{2} \sigma_p^2.
$$

ここで、$\gamma$ は収益率とリスクのバランスに関するパラメータです。本サンプルプログラムでは、以下で実装するポートフォリオ最適化ソルバのデフォルト設定として、$\gamma=1$ としており、以下の最適化では、このデフォルト設定を利用しています。リスクの低減をより重視する場合は、$\gamma$ の調整（増加）を行ってください。


In [ ]:
import amplify
import numpy as np
from typing import Any
import matplotlib.pyplot as plt


def calc_return_cov_rate(
    h: HistoricalDataLoader,  # ヒストリカルデータ
    purchase_prices: np.ndarray,  # 各銘柄の購入価額
    is_sampling_period: bool,  # 運用期間以前のヒストリカルデータに基づくか、運用期間中のデータに基づくか
) -> tuple[np.ndarray | None, np.ndarray | None]:
    """各金融商品における運用期間中の収益率（の平均）と共分散行列を計算"""
    if is_sampling_period:
        prices_end = h.extract_sampling_period().to_numpy()
        prices_start = np.roll(prices_end, -h.num_days_operation + 1, axis=0)
        stop = h.num_days_sampling - h.num_days_operation + 1
    else:
        # 運用期間中のヒストリカルデータに基づく場合、運用開始時の各銘柄の価額は購入価額を考慮
        # また、収益率のデータは一つ（つまり、共分散行列は無し）
        prices_end = h.extract_operation_period().to_numpy()
        # 運用期間中のデータがデータに含まれない場合
        if len(prices_end) == 0:
            return None, None
        prices_start = purchase_prices[np.newaxis, :]
        stop = 1
    return_rates = ((prices_end - prices_start) / prices_start)[0:stop]
    if len(return_rates) == 1 or len(return_rates[0]) == 1:
        return return_rates.mean(axis=0), None
    return return_rates.mean(axis=0), np.cov(return_rates, rowvar=False)


def calc_profit_risk(
    h: HistoricalDataLoader,  # ヒストリカルデータ
    w_i_percent: amplify.PolyArray | np.ndarray,  # 投資比率%
    purchase_prices: np.ndarray,  # 購入価額
    is_sampling_period=True,  # 運用期間以前のヒストリカルデータに基づくか、運用期間中のデータに基づくか
) -> tuple[Any, Any]:
    """与えられたポートフォリオ全体の収益率と分散を計算・返却"""
    w_i = w_i_percent / 100
    return_rates, cov_rates = calc_return_cov_rate(
        h, purchase_prices, is_sampling_period
    )
    if return_rates is None:
        return None, None
    profit = amplify.einsum("i,i->", w_i, return_rates)
    if cov_rates is None:
        return profit, None
    risk = amplify.einsum("i,j,ij->", w_i, w_i, cov_rates)
    return profit, risk


def generate_objective(
    h: HistoricalDataLoader,  # ヒストリカルデータ
    w_i_percent: amplify.PolyArray,  # 投資比率%（決定変数）
    purchase_prices: np.ndarray,  # 購入価額
    gamma: float,  # 平均・分散モデルにおける gamma
) -> amplify.Poly:
    """平均・分散モデルと付加項に基づく目的関数を構築・返却"""
    profit, risk = calc_profit_risk(h, w_i_percent, purchase_prices)
    return -profit + gamma * 0.5 * risk


def generate_constraint(w_i_percent: amplify.PolyArray) -> amplify.Constraint:
    """投資比率の合計が100%になる制約条件を構築・返却"""
    return amplify.equal_to(w_i_percent, 100)


def solve(
    h: HistoricalDataLoader,  # ヒストリカルデータ
    purchase_prices: np.ndarray,  # 購入価額
    gamma=1,  # 平均・分散モデルにおける gamma
    constraint_weight: float = 1,  # 制約条件に対する重み
    timeout_ms: int = 5000,  # 求解のタイムアウト (ms)
    num_solves: int = 3,  # 求解の直列実行回数 (https://amplify.fixstars.com/ja/docs/amplify/v1/serial.html)
    plot_optim_history=False,
) -> np.ndarray:
    """ヒストリカルデータ及び各商品の想定購入価額、運用予算に基づき、ポートフォリオ最適化を行う関数"""
    gen = amplify.VariableGenerator()

    # q は投資比率 %、分散投資の観点から各銘柄は最大 50% まで購入可能
    q = gen.array("Integer", len(h.tickers), bounds=(0, 50))

    objective = generate_objective(h, q, purchase_prices, gamma)
    constraint = generate_constraint(q)
    model = amplify.Model(objective, constraint_weight * constraint)

    client = amplify.FixstarsClient()
    client.parameters.timeout = datetime.timedelta(milliseconds=timeout_ms)
    client.parameters.outputs.num_outputs = 0  # 発見した全ての解を返す
    # client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # ローカル環境等で使用する場合、Fixstars Amplify AE のアクセストークンを入力してください。

    # 求解を実施
    result = amplify.solve(model, client, num_solves=num_solves)

    if len(result) == 0:
        print("no solution is found")

    if plot_optim_history:
        optim_history(result)

    return q.evaluate(result.best.values)


def optim_history(result: amplify.Result) -> None:
    """最適化の履歴を表示する関数"""
    ax = plt.figure().add_subplot()
    # それぞれの解の時刻と目的関数の値を取得
    for i in range(result.num_solves):
        times = [solution.time.total_seconds() for solution in result.split[i]]
        objectives = [solution.objective for solution in result.split[i]]
        ax.plot(times, objectives, marker="*")
    ax.set_xlabel("elapsed time in seconds")
    ax.set_ylabel("objective value")
    ax.grid(True)
    plt.show()

<a id="3"></a>

## 最適化の実行と評価

それでは、手始めに NASDAQ100 を構成する銘柄を対象とし、最適ポートフォリオを構築します。ここで、NASDAQ100 インデックスは単に各構成銘柄の時価総額に応じたポートフォリオとなっています。構築した最適ポートフォリオと NASDAQ100 インデックスの利益率を比較してみましょう。

問題設定として、ポートフォリオ最適化に使うヒストリカルデータは、運用開始日からさかのぼって 10 営業日分を使用し、運用期間は 5 営業日とします（これらの値は、ヒストリカルデータクラス `HistoricalDataLoader` のコンストラクタのデフォルト値として設定されています）。

運用開始日として、運用期間後の検証を行うため、ここでは 2024 年 4 月 1 日とします。通常のポートフォリオ最適化では、運用開始日は最適化したポートフォリオでの運用を開始する日となります。

上で定義した `HistoricalDataLoader` では、考慮する全ての銘柄が取引可能な日を営業日として定義し、営業日でない日付のヒストリカルデータは `HistoricalDataLoader` クラスのインスタンス時に予め削除しています。

### ヒストリカルデータの取得

それでは、`HistoricalDataLoader` を使って NASDAQ100 を構成する銘柄のヒストリカルデータを取得してみましょう。


In [ ]:
# ヒストリカルデータの保存場所
history_data_dir = pathlib.Path("../../../storage/portfolio_stooq")

# 運用開始日
date_of_operation = datetime.date.fromisoformat("2024-04-01")

# NASDAQ100 の構成銘柄のティッカー（2024年3月18日版）
nasdaq100_stocks = ["ADBE", "ADP", "ABNB", "GOOGL", "GOOG", "AMZN", "AMD", "AEP", "AMGN", "ADI",
                     "ANSS", "AAPL", "AMAT", "ASML", "AZN", "TEAM", "ADSK", "BKR", "BIIB", "BKNG",
                     "AVGO", "CDNS", "CDW", "CHTR", "CTAS", "CSCO", "CCEP", "CTSH", "CMCSA", "CEG",
                     "CPRT", "CSGP", "COST", "CRWD", "CSX", "DDOG", "DXCM", "FANG", "DLTR", "DASH",
                     "EA", "EXC", "FAST", "FTNT", "GEHC", "GILD", "GFS", "HON", "IDXX", "ILMN",
                     "INTC", "INTU", "ISRG", "KDP", "KLAC", "KHC", "LRCX", "LIN", "LULU", "MAR",
                     "MRVL", "MELI", "META", "MCHP", "MU", "MSFT", "MRNA", "MDLZ", "MDB", "MNST",
                     "NFLX", "NVDA", "NXPI", "ORLY", "ODFL", "ON", "PCAR", "PANW", "PAYX", "PYPL",
                     "PDD", "PEP", "QCOM", "REGN", "ROP", "ROST", "SIRI", "SBUX", "SNPS", "TTWO",
                     "TMUS", "TSLA", "TXN", "TTD", "VRSK", "VRTX", "WBA", "WBD", "WDAY", "XEL", "ZS"]  # fmt: skip

# ヒストリカルデータクラスをインスタンス化
nasdaq100_stocks_history = HistoricalDataLoader(
    nasdaq100_stocks,
    date_of_operation,
    history_data_dir,
    stdout=True,
)

試しに、上で作成したヒストリカルデータクラスのインスタンス `nasdaq100_stocks_history` を使って、アドビ社 (`"ADBE"`) のヒストリカルデータを表示してみましょう。運用開始日の前営業日から遡ること 10 営業日分のデータが表示されています。


In [ ]:
nasdaq100_stocks_history.extract_sampling_period()["ADBE"]

### 購入価額の決定

市場の取引時間中、銘柄の値は刻一刻と変化します。指値であっても成行であっても、購入価額は実際に購入するまで分かりませんが、購入価額を考慮しないことには最適なポートフォリオにおいてどの銘柄をいくつ買えば良いかの判断またそのポートフォリオの評価を行うことはできません。ここでは少し高めで購入することとし、運用開始日の市場において、前日終値に対し 0 ～ 1% 増の範囲からランダムに決定した費用で各銘柄の買付ができると仮定します。また、この割り増し分には各種手数料なども考慮されているものとします。


In [ ]:
rng = np.random.default_rng(0)


def calc_purchase_prices(h: HistoricalDataLoader, low=1.0, high=1.01) -> np.ndarray:
    """対象銘柄の購入価額を決定する関数。運用開始日前日の終値に対し、[low, high] の範囲での割増価額。"""
    # 運用開始日前日の終値（運用開始日前日＝サンプリング期間最終日）
    close = h.extract_sampling_period()[h.sample_end_date : h.sample_end_date].iloc[0]
    return close.to_numpy() * rng.uniform(low, high, size=len(close))


# 運用開始日の購入価額
purchase_prices_stocks = calc_purchase_prices(nasdaq100_stocks_history)

### 最適ポートフォリオの求解

それでは、上記で実装したソルバーを用いて、ヒストリカルデータに基づく最適ポートフォリオを求解します。


In [ ]:
solution = solve(nasdaq100_stocks_history, purchase_prices_stocks)

### ポートフォリオの評価

それでは得られたポートフォリオを評価します。以下の `evaluate()` では、評価対象のポートフォリオに対して、① ヒストリカルデータに対する収益率及びリスク、② 運用後の収益率（運用期間中のデータが与えられる場合）、を計算します。また、ここで、売却益への課税分 (日本国内における分離課税 20%) を引いた最終的な利益率を戻り値として返却します。


In [ ]:
import math

TAX_RATE = 0.2


def evaluate(
    sol: np.ndarray,
    purchase_prices: np.ndarray,
    h: HistoricalDataLoader,
    show_historical=False,
    tax_rate=TAX_RATE,
) -> float | None:
    """ポートフォリオでの運用結果を評価し、ヒストリカルデータに基づくポートフォリオ全体の収益率と
    （データが存在すれば）運用期間中の収益率を返却"""

    portfolio = {
        ticker: f"{int(sol[i])}%" for i, ticker in enumerate(h.tickers) if sol[i] > 0
    }
    print(f"portfolio: {portfolio}")

    # ヒストリカルデータに対する収益率及びリスク
    h_profit, h_risk = calc_profit_risk(h, sol, purchase_prices)
    if show_historical:
        print(f"historical profit: {float(h_profit)*100:.1f} %")
        if h_risk is not None:
            # 表示のリスクは、標準偏差値
            print(f"historical risk: {math.sqrt(float(h_risk))*100:.1f} %")

    op_profit, _ = calc_profit_risk(h, sol, purchase_prices, is_sampling_period=False)

    if op_profit is None:
        return None

    # 運用後の収益率（運用期間中のデータが与えられる場合のみ、収益に対する課税 tax_rate も考慮）
    print(f"operational profit: {float(op_profit)*100:.1f} %")
    return (
        float(op_profit) * (1 - tax_rate) if float(op_profit) > 0 else float(op_profit)
    )


# 上のセルで求解した最適ポートフォリオを評価
return_rate = evaluate(
    solution,
    purchase_prices_stocks,
    nasdaq100_stocks_history,
    show_historical=True,
)

今回構築したポートフォリオは、NASDAQ100 の構成銘柄から構成されています。同じ構成銘柄で、時価総額に比例して構成されている NASDAQ100 インデックスの評価を行い、比較してみましょう。ポートフォリオ構築条件と同じ運用開始日、運用期間を対象とします。


In [ ]:
nasdaq100_index_history = HistoricalDataLoader(
    ["^NDX"],  # NASDAQ100 インデックスのティッカー
    date_of_operation,
    history_data_dir,
)

# 運用開始日の購入価額
purchase_prices_index = calc_purchase_prices(nasdaq100_index_history)

# 1 銘柄（インデックス）のみなので、求解することなく投資割合 100% で評価
return_rate = evaluate(
    np.array([100]),
    purchase_prices_index,
    nasdaq100_index_history,
    show_historical=True,
)

<a id="4"></a>

## ポートフォリオ最適化ソルバによる運用シミュレーション

それでは、本サンプルプログラムで実装したポートフォリオ最適化ソルバーを用いて、運用シミュレーションを行います。運用開始日として、2024 年 1 月 1 日以降の最初の営業日とし、この日からヒストリカルデータのサンプル期間ごとに 5 回運用を行います。運用・最適化条件は、`HistoricalDataLoader` のデフォルト値として設定されている通りで、運用期間は `num_days_operation = 5` 営業日、各運用日に行う最適化で用いるヒストリカルデータとして各運用日から遡った `num_days_sampling = 10` 営業日分を用います。各投資ラウンドで構築するポートフォリオに基づき、前回投資分で増減した資金を複利も含めて運用します。以下の図はこのような運用の流れの模式図です。

![operation_flow](../figures/portfolio_flow.drawio.svg)


In [ ]:
def simulate_operation(
    tickers: list[str],  # 運用時に考慮する銘柄のティッカー
    operation_date: datetime.date | list[datetime.date],  # 運用開始日
    num_operation_rounds: int,  # 運用ラウンド数
) -> tuple[list[datetime.date], list[float]]:
    """num_operations 回の投資ラウンドに渡って売買する運用シミュレーションを実施"""

    # ヒストリカルデータを走査して、各運用ラウンドの運用開始日と終了日を operation_dates に格納
    operation_dates: list[datetime.date] = []
    if isinstance(operation_date, list):
        operation_dates = operation_date
    else:
        i_operation_round = 0
        while i_operation_round < num_operation_rounds:
            while True:
                h = HistoricalDataLoader(tickers, operation_date, history_data_dir)
                # operation_date が営業日でない場合、1日後を営業日として再検証
                if not h.exist_operation_duration():
                    operation_date += ONE_DAY
                    break
                i_operation_round += 1
                operation_dates.append(operation_date)
                operation_dates.append(h.operation_end_date)
                operation_date += datetime.timedelta(days=h.num_days_sampling)
                break

    # 各運用ラウンドにおける運用開始・終了時の資産額を入れるリスト
    fund_evolution: list[float] = []
    funds = 1.0  # 初期資産額
    print(f"Initial funds: {funds:.2f}")

    for i, date in enumerate(operation_dates):
        # i が奇数の時、date は運用終了日なので、スキップ
        if i % 2 == 1:
            continue

        h = HistoricalDataLoader(tickers, date, history_data_dir)

        print("============================================")
        print(f"Operation round: {int(i / 2) + 1} of {num_operation_rounds}, {date}")

        # 当運用ラウンド開始日における資産額を fund_evolution に追加
        fund_evolution.append(funds)

        # 当運用ラウンド開始日における全銘柄の購入価額
        prices = calc_purchase_prices(h)

        # 銘柄候補が複数の場合、最適ポートフォリオの求解、1つの場合、その銘柄を 100% 購入
        if len(prices) > 1:
            solution = solve(h, prices)
        else:
            solution = np.array([100])

        # 当ラウンドにおける運用結果を評価し、増減後の資産額を計算
        operational_return = evaluate(solution, prices, h)
        if operational_return is not None:
            funds *= 1.0 + operational_return
            print(f"Current funds: {funds:.2f}")

        # 当運用ラウンド終了日における資産額を fund_evolution に追加
        fund_evolution.append(funds)

    return operation_dates, fund_evolution

In [ ]:
start_date = datetime.date.fromisoformat("2024-01-01")  # 初期運用開始日
num_operation_rounds = 5  # 投資ラウンド数

operation_dates, optim_portfo_evolution = simulate_operation(
    nasdaq100_stocks, start_date, num_operation_rounds
)

最適ポートフォリオではなく、NASDAQ100 インデックスに基づく銘柄の売買を上記の運用ラウンドと同日に実施する運用の場合のシミュレーションは以下になります。


In [ ]:
_, index_op_evolution = simulate_operation(["^NDX"], start_date, num_operation_rounds)

また、初回運用ラウンドにおける運用開始日に NASDAQ100 インデックスに基づいて銘柄購入を行い、最終運用ラウンドにおける運用終了日まで売買を行わない場合のシミュレーションは以下になります。


In [ ]:
round_start_date = operation_dates[0]
round_end_date = operation_dates[-1]

h = HistoricalDataLoader(["^NDX"], round_start_date, history_data_dir)

# 第一回の運用ラウンド開始日における購入価額
prices = calc_purchase_prices(h)

# 最終運用ラウンド最終日に相当するインデックス
i_end = h.history.index.get_loc(round_end_date.strftime("%Y-%m-%d"))

# 最終運用ラウンド最終日に売却後の課税済み資金
return_rate = h.history.iloc[i_end].iloc[0] / prices[0]
if return_rate > 1:
    return_rate = (return_rate - 1) * (1 - TAX_RATE) + 1
index_no_op_evolution = [1, return_rate]

最後に、NASDAQ100 インデックスの構成銘柄を、

- `optim portfo`：最適ポートフォリオに従って運用した場合
- `index-op`：インデックスに基づいて運用した場合
- `index-no-op`：インデックスに基づいて最後まで売買しなかった場合

の 3 通りの運用結果をプロットします。

ここで注意点として、銘柄の購入では、ランダムに 0~1% の割増価額での購入を行っているので、各運用ラウンドごとに売買を繰り返す `optim portfo` と `index-op` では、本サンプルプログラムで考慮した割増額や課税の (悪い) 影響がその都度入ることになります。最適ポートフォリオに基づいて行う積極的な金融商品の売買は、市場が下落傾向にあるときには特に不利に働く事に注意してください。

本サンプルプログラムでは、ヒストリカルデータのサンプリング期間や運用期間等は、ヒストリカルデータクラス `HistoricalDataLoader` のコンストラクタのデフォルト値として、それぞれ 10 日と 5 日が設定されています。ポートフォリオ全体として期間の価格変動が比較的小さい場合などは、運用期間などを大きくとることで、売買回数を減らすことが可能です。

また、株式以外のより幅広い金融商品（例：金属、化石燃料、暗号通貨）などもポートフォリオへ加えることで、より低リスク・高リターンを期待することができます。例えば、本サンプルプログラムで利用の Stooq では、金は `GC.C`、原油は `CL.C`、ビットコイン/米ドルは `BTCUSD` というティッカーで取得することができます。様々な商品をポートフォリオへ加えて、運用シミュレーションを実施してみましょう。


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import dates as mdates

ax = plt.figure().add_subplot()
color_stocks = [0.2, 0.2, 1]
color_index = [1, 0.2, 0.2]

# "optim portfo", "index-op" をプロット（運用ラウンド分）
operation_dates = np.array(operation_dates)
ax.plot(operation_dates, optim_portfo_evolution, linestyle=":", color=color_stocks)
ax.plot(operation_dates, index_op_evolution, linestyle=":", color=color_index)
for i, date in enumerate(operation_dates):
    text = "E"
    if i % 2 == 0:
        text = "S"
        (optimal,) = ax.plot(
            operation_dates[i : i + 2],
            optim_portfo_evolution[i : i + 2],
            color=color_stocks,
            marker="*",
        )
        (index,) = ax.plot(
            operation_dates[i : i + 2],
            index_op_evolution[i : i + 2],
            color=color_index,
            marker="*",
        )
    ax.annotate(text, (operation_dates[i], optim_portfo_evolution[i]), fontsize=10)
    ax.annotate(text, (operation_dates[i], index_op_evolution[i]), fontsize=10)

# index-no-op をプロット（初回運用ラウンド初日と最終運用ラウンド最終日のみ）
(no_transact,) = ax.plot(
    [operation_dates[0], operation_dates[-1]],
    index_no_op_evolution,
    marker="o",
    color=color_index,
)

ax.legend(
    [optimal, index, no_transact],
    ["optim portfo", "index-op", "index-no-op"],
    loc="lower right",
)
ax.annotate(
    " S: Start date of round\n E: End date",
    (operation_dates[0], max(optim_portfo_evolution) * 0.99),
    fontsize=10,
)
ax.set_xlabel("Date", fontsize=10)
ax.set_ylabel("Total asset", fontsize=10)
ax.tick_params(labelsize=10)
ax.set_xlim((operation_dates[0] - ONE_DAY, operation_dates[-1] + ONE_DAY))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m/%d"))

plt.show()